In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

In [ ]:
path = "/mnt/c/Users/olivi/OneDrive - Amherst College/6 Summer 2025/surface-detection"

##### 128x2 Variable Threshold Sequence Dataset

In [ ]:
thresholds = [
    [25, 26, 27, 28, 28],
    [21, 17, 21, 20, 21],
    [19, 14, 22, 25, 25],
    [25, 22, 27, 24, 27],
    [24, 22, 24, 26, 25]
]

s1_counts = [
    [156, 145, 154, 160, 142], # User 1
    [149, 130, 156, 136, 120], # User 2
    [142, 103, 142, 152, 137], # User 3
    [145, 144, 159, 149, 142], # User 4
    [137, 138, 136, 124, 134]  # User 5
]

locations = ["A", "B", "C", "D", "E"]
labels = ["User 1", "User 2", "User 3", "User 4", "User 5"]

In [ ]:
s2_counts = [
    [156, 147, 154, 160, 143], # User 1
    [158, 125, 155, 137, 122],
    [147, 100, 145, 153, 140],
    [150, 153, 161, 150, 148],
    [149, 143, 147, 157, 140]
]

In [ ]:
def dataset_barplot(split, title=""):
    colors = plt.get_cmap('Set2').colors

    split = np.array(split)
    x = np.arange(5)

    fig, ax = plt.subplots()
    bottom = np.zeros(5)
    for i, row in enumerate(split):
        ax.bar(x, row, bottom=bottom, label=labels[i], color = colors[i])
        bottom += row 

    ax.set_xticks(x)
    ax.set_xticklabels(locations)
    ax.set_xlabel("Location")
    ax.set_ylabel("Number of Sequences")

    # ax.legend(loc="upper center", bbox_to_anchor=(0.5, 1.12), ncol=5)
    pos = ax.get_position()
    ax.set_position([pos.x0, pos.y0, pos.width * 0.9, pos.height])
    ax.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))
    
    plt.title(title)
    plt.show()

In [ ]:
# counts: stacked barplot
dataset_barplot(s2_counts)

# thresholds: line plot
colors = plt.get_cmap('Set3').colors
for i in range(5):
    plt.plot(locations, thresholds[i], label=labels[i], color = colors[i])
plt.legend()
plt.xlabel("Location")
plt.ylabel("Peak Acceleration Threshold")
plt.show()

##### Single sequence (normalized)

In [ ]:
def sequence_norm(user=1, location="A", instance=1):
    A1 = pd.read_csv(f"{path}/sequences/varied-threshold/User{user}_Location{location}_Normal_{instance}.dat", header=None)
    plt.plot(list(range(1, 129)), A1[0] / A1[0].max(), label='acceleration')
    plt.plot(list(range(1, 129)), A1[1] / A1[1].max(), color='red', label="Gyroscope")
    plt.title("Normalized Accelerometer, Gyroscope for Sequence")
    plt.legend()
    plt.show()

sequence_norm()

##### Dataset split visualization

In [ ]:
# load dataset partitions, filenames
lines = []
with open(f"{path}/sequences/filenames.txt", 'r') as filenames:
    lines = filenames.readlines()

In [ ]:
def split_barplot(filepath, title):
    indices = []
    with open(filepath) as f:
        indices = f.readlines()
    counts = [[0 for i in range(0, 5)] for j in range(0, 5)] # initialize to zeros
    for i in indices:
        user = 0
        match lines[int(i.strip())-1].split('_')[0]:
            case "User1" : user = 0
            case "User2" : user = 1
            case "User3" : user = 2
            case "User4" : user = 3
            case "User5" : user = 4
        match lines[int(i.strip())-1].split('_')[1]:
            case "LocationA" : counts[user][0] += 1
            case "LocationB" : counts[user][1] += 1
            case "LocationC" : counts[user][2] += 1
            case "LocationD" : counts[user][3] += 1
            case "LocationE" : counts[user][4] += 1
    dataset_barplot(counts, title)

In [ ]:
split_barplot(f"{path}/logs/dataset-info/u5out_train.txt", "Training Set")
split_barplot(f"{path}/logs/dataset-info/u5out_val.txt", "Validation Set")
split_barplot(f"{path}/logs/dataset-info/u5out_test.txt", "Test Set")

##### Confusion heatmap

In [ ]:
# confusion matrix heatmap, normalized over true (rows)
def confusion_heatmap(model="ver6", fsetname="train", setname="Training", dataset=1):
    set= pd.read_csv(f"{path}/logs/set-results/{model}_accuracy_{fsetname}_set{dataset}.txt")
    sklearn.metrics.ConfusionMatrixDisplay.from_predictions(set["True"], set["Predicted"], 
                                                            cmap='cividis', 
                                                            display_labels=["A", "B", "C", "D", "E"],
                                                            normalize="true")
    plt.xlabel("Predicted location")
    plt.ylabel("True location")
    plt.title(f"{model.capitalize()} Predictions on {setname.capitalize()} Set (Set {dataset})")
    plt.savefig(f"{path}/figures/set-results/{model}_confusion_{fsetname}_set{dataset}.png")
    # plt.show()

In [ ]:
model = "ver7" 
set = 2
confusion_heatmap(model, "train", "training", set)
confusion_heatmap(model, "val", "validation", set)
confusion_heatmap(model, "test", "test", set)

##### Accuracy, Precision and Recall Comparisons

In [ ]:
versions = ["ver2", "ver3", "ver4", "ver5a", "ver5b", "ver6", "ver7"]
sets = ["train", "val", "test"]
metrics = ["accuracy", "precision", "recall"]
trials = [1, 2]

In [ ]:
def comparison(metric="accuracy", trial=1):
    x = ["A", "B", "C", "D", "E"]

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
    for k in range(len(versions)):
        model = versions[k]
        y = []
        if model == "ver2" and trial == 1:
            continue
        for i in range(len(sets)):
            guesses = pd.read_csv(f"{path}/logs/nn-results/{model}_accuracy_{sets[i]}_{trial}.txt")
            confusion = sklearn.metrics.confusion_matrix(guesses["True"], guesses["Predicted"])
            if metric == "precision":
                # precision = TP / (TP + FP)
                y.append([confusion[j][j] / np.sum(confusion[:, j]) for j in range(len(x))])
            elif metric == "recall":
                # recall = TP / (TP + FN)
                y.append([confusion[j][j] / np.sum(confusion[j]) for j in range(len(x))])
            else:
                # accuracy = true predictions / all predictions
                y.append([np.trace(confusion) / np.sum(confusion) for j in range(len(x))])

        colors = plt.get_cmap('Set2').colors

        ax1.plot(x, y[0], label=model, color=colors[k])
        ax2.plot(x, y[1], label=model, color=colors[k])
        ax3.plot(x, y[2], label=model, color=colors[k])

        if trial == 1:
            trialname = "Even"
        else:
            trialname = "U5 Separate"
    
    ax1.set_ylabel(metric.capitalize())
    ax2.set_xlabel("Location")
    ax2.set_title(f"{metric.capitalize()} on Training, Validation, and Test Sets ({trialname})")
    ax3.legend(loc='center right', bbox_to_anchor=(1.34, 0.5))
    plt.tight_layout()
    plt.savefig(f"{path}/figures/nn-results/{metric}_{trial}.png")
    # plt.show()

In [ ]:
comparison("accuracy", 1)
comparison("accuracy", 2)
comparison("recall", 1)
comparison("recall", 2)
comparison("precision", 1)
comparison("precision", 2)

##### Recall vs. Class Counts

In [ ]:
s1_counts = [729, 660, 747, 721, 675]
versions = ["ver2", "ver3", "ver4", "ver5a", "ver5b", "ver6", "ver7"]
sets = ["train", "val", "test"]

# count true positives across versions
tps = [[0, 0, 0, 0, 0] for i in range(len(sets))]
for i in range(len(versions)):
    model = versions[i]
    for j in range(len(sets)):
        guesses = pd.read_csv(f"{path}/logs/nn-results/{model}_accuracy_{sets[j]}_2.txt")
        confusion = sklearn.metrics.confusion_matrix(guesses["True"], guesses["Predicted"], normalize="true")
        for k in range(5):
            tps[j][k] += confusion[k][k]

# calculate average and recall (TP / (TP + FN))
for i in range(len(sets)):
    for j in range(5):
        tps[i][j] = (tps[i][j] / len(versions))

# plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
ax1.scatter(s1_counts, tps[0])
ax2.scatter(s1_counts, tps[1])
ax3.scatter(s1_counts, tps[2])

# labels
ax1.set_title("Training Set")
ax2.set_title("Validation Set")
ax3.set_title("Test Set")
ax1.set_ylabel("Recall")
ax2.set_xlabel("Items in Class")